## 모델 파인튜닝

### 데이터셋

In [1]:
!pwd

/kaggle/working


In [5]:
!ls -la ../input/test-train-datasets/datasets

total 50680
drwxr-xr-x 2 nobody nogroup        0 Feb 10 03:35 .
drwxr-xr-x 3 nobody nogroup        0 Feb 10 03:35 ..
-rw-r--r-- 1 nobody nogroup   130329 Feb 10 03:35 test.csv
-rw-r--r-- 1 nobody nogroup 51762590 Feb 10 03:35 train.csv


In [6]:
!mkdir datasets

In [7]:
cp ../input/test-train-datasets/datasets/train.csv ./datasets

## 모델 관련

In [8]:
!pip install autotrain-advanced==0.7.77 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 79.4 MB/s eta 0:00:00:00:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14

In [9]:
base_model = 'beomi/Yi-Ko-6B' # huggingface에 있는 원본 모델
finetuned_model = 'yi-ko-6b-text2sql' # 교육 후 모델

# 셋팅 옵션 리스트
# --model {base_model} \
# --project-name {finetuned_model} \
# --data-path
# --text-column text \

# --use-peft 양자화 옵션
!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path datasets/ \
--text-column prompt \
--lr 2e-4 \
--batch-size 2 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft

INFO     | 2025-02-10 03:51:02 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2025-02-10 03:51:02 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: version, train, inference, deploy, backend, config, func
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 361568.36 exa
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 396982.09 exa
INFO     | 2025-02-10 03:51:05 | autotrain.backend:create:300 - Starting local training...
INFO     | 2025-02-10 03:51:05 | autotrain.commands:launch_command:327 - ['accelerate', 'launch', '--multi_gpu', '--num_machines', '1', '--num_processes', '2', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'yi-ko-6b-text2sql/training_params.json']
INFO     | 2025-02-10 03:51:05 | autotrain.commands:launch_command:328 - {'model': 'beomi/Yi-Ko-6B', 'project_name': 'yi-ko-6b-text2sql', 'data_path': 'yi-ko-6b-text2sql/autotrain-data', 'train_split': 'train', 'valid_spl

### 기존 모델을 새로운 데이터셋으로 튜닝